In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q https://mirrors.estointernet.in/apache/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz -P /content/drive/MyDrive # link wrong in blog
!tar xf /content/drive/Shareddrives/ds256-2022/spark-3.0.3-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop2.7"

Mounted at /content/drive


In [ ]:
!pip install graphframes

     |████████████████████████████████| 154 kB 5.0 MB/s 


In [ ]:
os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages graphframes:graphframes:0.8.1-spark3.0-s_2.12 pyspark-shell"
import findspark
findspark.init()
findspark.find()
from pyspark.sql import SparkSession
spark = SparkSession.builder\
         .master("local")\
         .appName("Colab")\
         .config('spark.ui.port', '4050')\
         .config("spark.jars.packages", "graphframes:graphframes:0.8.0-spark2.4-s_2.11")\
         .getOrCreate()
spark

In [ ]:
!pip install python-igraph

In [ ]:
!pip install cairocffi

In [ ]:
#Create Data Frames
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.functions import col, concat
from pyspark.sql.functions import lit
from pyspark.sql.functions import expr
import sys
import json
import networkx as nx
import matplotlib.pyplot as plt


sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)
from graphframes import *

**CREATE DATAFRAME**

In [ ]:
import csv
from os import listdir, getcwd
from pyspark.sql import Row

classes = [['dynamic', ['Comment', 'Forum', 'Person', 'Post']], ['static', ['Organisation', 'Place', 'Tag', 'TagClass']]]

dataSetPath = "/content/drive/MyDrive/property_graph_dataset/singular-projected-fk/"

arrayOfVertices = []
for i in classes:
  folder = i[0]
  for Class in i[1]:
    filenames = listdir(dataSetPath + folder + '/' + Class)
    fileName =  [ filename for filename in filenames if filename.endswith( '.csv' ) ][0]
    fileObj = open(dataSetPath + folder + '/' + Class + '/' + fileName)
    for row in csv.DictReader(fileObj, skipinitialspace=True, delimiter="|"):
      diction = {k: v for k, v in row.items()}
      diction['class'] = Class
      arrayOfVertices.append(diction)

vertexRDD = sc.parallelize(arrayOfVertices)
vertexDF = sqlContext.read.json(vertexRDD)
vertexDF = vertexDF.withColumn("cid",col("id")).withColumn("id",concat(col("id"),lit(','),col("class")))
vertexDF.show()

arrayOfEdges = []
for folder in ['static', 'dynamic']:
  relation_folders = listdir(dataSetPath + folder)
  for relation_folder in relation_folders:
    split_folder_list = relation_folder.split("_")
    if len(split_folder_list) != 3:
      continue
    else:
      sourceClass, relation, destClass = split_folder_list

    filenames = listdir(dataSetPath + folder + '/' + relation_folder)
    fileName =  [ filename for filename in filenames if filename.endswith( '.csv' ) ][0]
    fileObj = open(dataSetPath + folder + '/' + relation_folder + '/' + fileName)
    for row in csv.DictReader(fileObj, skipinitialspace=True, delimiter="|"):
      diction = {k: v for k, v in row.items()}
      if sourceClass == destClass:
        diction["sourceID"] = diction.pop(sourceClass + '1Id')
        diction["destID"] = diction.pop(destClass + '2Id')
      else:
        diction["sourceID"] = diction.pop(sourceClass + 'Id')
        diction["destID"] = diction.pop(destClass + 'Id')
      diction['sourceClass'] = sourceClass
      diction['destClass'] = destClass
      diction['edgeRelation'] = relation

      arrayOfEdges.append(diction)

edgeRDD = sc.parallelize(arrayOfEdges)
edgeDF = sqlContext.read.json(edgeRDD)
edgeDF = edgeDF.withColumn("src",concat(col("sourceID"),lit(','),col("sourceClass"))).withColumn("dst",concat(col("destID"),lit(','),col("destClass"))).drop("sourceID").drop("destID")
edgeDF.show(20,False)

+---------------+--------+-----------------+-------+--------------------+--------------------+---------+------+--------------------+---------+--------+--------+------+--------------+----+-----+----+----+-------------+
|_corrupt_record|birthday|      browserUsed|  class|             content|        creationDate|firstName|gender|                  id|imageFile|language|lastName|length|    locationIP|name|title|type| url|          cid|
+---------------+--------+-----------------+-------+--------------------+--------------------+---------+------+--------------------+---------+--------+--------+------+--------------+----+-----+----+----+-------------+
|           null|    null|          Firefox|Comment|                 yes|2012-07-08T16:48:...|     null|  null|962072674305,Comment|     null|    null|    null|     3|91.191.192.127|null| null|null|null| 962072674305|
|           null|    null|          Firefox|Comment|              thanks|2012-07-08T13:32:...|     null|  null|962072674306,Comm

In [ ]:
graph = GraphFrame(vertexDF, edgeDF)

In [51]:
def genpathquery(len):
  basev = "x"
  basee = "e"
  query = ""
  for i in range(len-1):
    query += "("+basev+str(i)+")-["+basee+str(i)+"]->("+basev+str(i+1)+");"
  query += "("+basev+str(len-1)+")-["+basee+str(len-1)+"]->("+basev+str(len)+")"
  return query

# **MAKE CLASS FOR INPUT**

In [52]:
class component:
  def __init__(self, relation=None, next=None, type="vertex", conditions=[], v_name="", e_name=""):
    self.next = next
    self.relation = relation #if relation = None edges with all relations will be included
    self.type = type #if it is a single vertex or series of vertex
    self.conditions = conditions #conditions on vertices for attributes
    self.v_name = v_name
    self.e_name = e_name

def makeComponentFromArr(arr):
  path_query = ''
  vertex_name, conditions, edge_name, relation = arr[0]
  prev_component = component(relation=relation, next=None, conditions=conditions, v_name=vertex_name, e_name=edge_name)
  head_component = prev_component

  for element in arr[1:]:
    vertex_name, conditions, edge_name, relation = element
    curr_component = component(relation=relation, next=None, conditions=conditions, v_name=vertex_name, e_name=edge_name)
    prev_component.next = curr_component
    prev_component = curr_component
  #print(vars(head_component), vars(head_component.next), vars(head_component.next.next))
  return head_component

first_component = makeComponentFromArr([["v1", [["v1","class","Person"]], "e1", ["e1","likes"]], 
                                        ["v2", [["v2","class", "Comment" ]], "e2", ["e2","hasCreator"]], 
                                        ["v3", [["v3","class","Person"],["v3","cid","2199023255557"]], "", "r3"]])

def merge_components(*components, edge_names=[], relations=[]):
  head_component = components[0]
  n = len(components)
  if len(edge_names) == 0 and len(relations) == 0:
    edge_names = [""]*(n-1)
    relations = [""]*(n-1)
  if len(edge_names) != n-1 or len(relations) != n-1:
    raise ValueError("Length of edge names and relations should be number of components-1")
  else:
    for i in range(n-1):
      comp1 = components[i]
      comp2 = components[i+1]

      while comp1.next != None:
        comp1 = comp1.next

      comp1.next = comp2
      comp1.e_name = edge_names[i]
      comp1.relation = relations[i]

    return head_component


one_component = makeComponentFromArr([["v1", [], "e1", "Likes"]])
two_component = makeComponentFromArr([["v1", [], "e1", "Likes"]])
new_component = merge_components(one_component, two_component, edge_names=["E"], relations=["R"])
#print(vars(new_component), vars(new_component.next))

def get_query_string_without_conditions(component):
  query = ""
  while component.next != None:
    query += "("+ component.v_name + ")-[" + component.e_name + "]->"
    component = component.next
    query += "("+ component.v_name + ");"
  query = query[:-1]
  return query

get_query_string_without_conditions(first_component)

def get_query_string_with_conditions(component):
  query = ""
  vertex_conditions = [] # n vertices
  edge_relations = [] # n-1 edges

  
  
  while component.next != None:
    vertex_conditions.append(component.conditions)
    edge_relations.append(component.relation)
    query += "("+ component.v_name + ")-[" + component.e_name + "]->"
    component = component.next
    query += "("+ component.v_name + ");"
  query = query[:-1]
  vertex_conditions.append(component.conditions)
  
  return query, vertex_conditions, edge_relations

query = get_query_string_with_conditions(first_component)
motif = query[0]
vcond = query[1]
econd = query[2]

In [53]:
#Query 3
from pyspark.sql.functions import *
input = "2199023255557"
inputclass = "Person"
inp = input + "," + inputclass
path = graph.find(motif)
for i in econd:
  cond = i[0]+"."+"edgeRelation"
  path = path.filter(col(cond) == i[1])
for i in vcond:
  for j in i:
    cond = j[0]+"."+j[1]
    print(cond,j[2])
    path = path.filter(col(cond) == j[2])
output = path.withColumn("minutesLatency",(to_timestamp(col("e2.creationDate")).cast("long") - to_timestamp(col("v2.creationDate")).cast("long"))/60).select(col("v1.cid").alias("friend.id"),col("v1.firstName").alias("friend.firstName"),col("v1.lastName").alias("friend.lastName"),col("v1.creationDate").alias("likes.creationDate"),col("v2.cid").alias("message.id"),col("v2.content").alias("message.content"),col("v2.imageFile").alias("message.imageFile"),col("minutesLatency"))
output.show(20,False)

v1.class Person
v2.class Comment
v3.class Person
v3.cid 2199023255557
+-------------+----------------+---------------+-----------------------------+------------+----------------------------------------------------------------------------------+-----------------+--------------+
|friend.id    |friend.firstName|friend.lastName|likes.creationDate           |message.id  |message.content                                                                   |message.imageFile|minutesLatency|
+-------------+----------------+---------------+-----------------------------+------------+----------------------------------------------------------------------------------+-----------------+--------------+
|8796093022244|John            |Reddy          |2010-09-28T17:46:50.451+00:00|824633723066|About Marlene Dietrich, , she acted on thAbout A Day in the Life,  has been covere|null             |0.0           |
+-------------+----------------+---------------+-----------------------------+------------+-------

In [ ]:
#Query 1
input = ["2199023255557","Tissa"]
inputclass = "Person"
inp = input[0] + "," + inputclass
pathlenlb = 1
pathlenub = 3
qout = []
print(genpathquery(3))
path = graph.find(genpathquery(3))
path.show()
for j in range(pathlenlb-1,3):
  cname = "e"+str(j)+".edgeRelation"
  path = path.filter(col(cname) == "knows")
path = path.filter(col("x0.id") == inp)
selected = [s for s in path.columns if 'x' in s]
colstackstr = ""
for i in range(pathlenlb,pathlenub):
  colstackstr += "'"+str(i)+"',x"+str(i)+","
colstackstr += "'"+str(pathlenub)+"',x"+str(pathlenub)
output = path.select(selected).select(expr("stack("+str(pathlenub)+","+colstackstr+") as (distanceFromPerson,otherPerson)")).distinct()
output.select(output.otherPerson.cid,output.otherPerson.lastName,output.distanceFromPerson,output.otherPerson.birthday,output.otherPerson.creationDate,output.otherPerson.gender,output.otherPerson.browserUsed,output.otherPerson.locationIP,output.otherPerson.locationIP).show(20,False)

(x0)-[e0]->(x1);(x1)-[e1]->(x2);(x2)-[e2]->(x3)
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                  x0|                  e0|                  x1|                  e1|                  x2|                  e2|                  x3|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|[,, Firefox, Comm...|[, 2012-02-17T10:...|[,, Firefox, Comm...|[, 2012-02-16T20:...|[, 1989-08-05, Fi...|[, 2012-09-26T00:...|[,, Firefox, Post...|
|[,, Chrome, Comme...|[, 2012-02-17T15:...|[,, Firefox, Comm...|[, 2012-02-16T20:...|[, 1989-08-05, Fi...|[, 2012-09-26T00:...|[,, Firefox, Post...|
|[,,, Forum,, 2010...|[, 2011-01-04T22:...|[,, Firefox, Post...|[, 2011-01-04T22:...|[, 1989-08-05, Fi...|[, 2012-09-26T00:...|[,, Firefox, Post...|
|[,, Firefox, Comm...|[, 2011-07-22T04:...|[,, Firefox, Po

In [ ]:
#Query 2
from pyspark.sql.functions import unix_timestamp, from_unixtime
input = ("2199023255557","2012-12-30T00:00:00.000+05:30")
inputclass = "Person"
inp = input[0] + "," + inputclass

path = graph.find("(x1)-[e0]->(x0);(x2)-[e1]->(x0)")
path = path.filter(col("e0.edgeRelation") == "knows").filter(col("e1.edgeRelation") == "hasCreator")
path = path.filter(col("x0.class") == "Person").filter(col("x1.class") == "Person").filter((col("x2.class") == "Comment") | (col("x2.class") == "Post"))
path = path.filter(col("x1.id") == inp).filter(col("x2.creationDate") < input[1])
selected = [s for s in path.columns if 'x' in s]
output = path.select(selected).select(col("x0.cid").alias("friend.id"),col("x0.firstName").alias("friend.firstName"),col("x0.lastName").alias("friend.lastName"),col("x2.cid").alias("message.id"),col("x2.content").alias("message.content"),col("x2.imageFile").alias("message.imageFile"),col("x2.creationDate").alias("message.creationDate"))
output.show(20,False)

+--------------+----------------+---------------+-------------+-----------------------------------------------------------------------------------------------------------+----------------------+-----------------------------+
|friend.id     |friend.firstName|friend.lastName|message.id   |message.content                                                                                            |message.imageFile     |message.creationDate         |
+--------------+----------------+---------------+-------------+-----------------------------------------------------------------------------------------------------------+----------------------+-----------------------------+
|8796093022237 |Lei             |Zhang          |1099511629686|                                                                                                           |photo1099511629686.jpg|2012-09-19T15:09:24.124+00:00|
|8796093022237 |Lei             |Zhang          |824633721307 |LOL                                  